In [2]:
from sentence_transformers import SentenceTransformer
import pandas as pd
import numpy as np
import faiss

/Users/felix/MSE/03_projects/MT/06_rag/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
PARAGRAPHS = '../data/kilt_wiki_small_paragraphs.jsonl'
QUESTIONS = '../data/kilt_fever_train_small.jsonl'
MODEL = 'jinaai/jina-embeddings-v3'
INDEX = '../data/kilt_wiki_small.index'

In [3]:
df_q = pd.read_json(QUESTIONS, lines=True)
df_q

,id,input,meta,output
0,150448,Roman Atwood is a content creator.,"{'left_context': '', 'mention': '', 'right_con...","[{'answer': 'SUPPORTS', 'meta': {'score': -1},..."
1,156709,Adrienne Bailon is an accountant.,"{'left_context': '', 'mention': '', 'right_con...","[{'answer': 'REFUTES', 'meta': {'score': -1}, ..."
2,129629,Homeland is an American television spy thrille...,"{'left_context': '', 'mention': '', 'right_con...","[{'answer': 'SUPPORTS', 'meta': {'score': -1},..."
3,33078,The Boston Celtics play their home games at TD...,"{'left_context': '', 'mention': '', 'right_con...","[{'answer': 'SUPPORTS', 'meta': {'score': -1},..."
4,6744,The Ten Commandments is an epic film.,"{'left_context': '', 'mention': '', 'right_con...","[{'answer': 'SUPPORTS', 'meta': {'score': -1},..."
5,226034,Tetris has sold millions of physical copies.,"{'left_context': '', 'mention': '', 'right_con...","[{'answer': 'SUPPORTS', 'meta': {'score': -1},..."
6,76253,There is a movie called The Hunger Games.,"{'left_context': '', 'mention': '', 'right_con...","[{'answer': 'SUPPORTS', 'meta': {'score': -1},..."
7,138503,"Stranger Things is set in Bloomington, Indiana.","{'left_context': '', 'mention': '', 'right_con...","[{'answer': 'REFUTES', 'meta': {'score': -1}, ..."
8,129983,Ryan Seacrest is a person.,"{'left_context': '', 'mention': '', 'right_con...","[{'answer': 'SUPPORTS', 'meta': {'score': -1},..."
9,73170,Puerto Rico is not an unincorporated territory...,"{'left_context': '', 'mention': '', 'right_con...","[{'answer': 'REFUTES', 'meta': {'score': -1}, ..."


In [6]:
df_p = pd.read_json(PARAGRAPHS, lines=True)
df_p.reset_index()

,index,kilt_id,wikipedia_id,wikipedia_title,paragraph,paragraph_id
0,0,76383,76383,The Ten Commandments (1956 film),The Ten Commandments (1956 film)\n,0
1,1,76383,76383,The Ten Commandments (1956 film),The Ten Commandments is a 1956 American epic r...,1
2,2,76383,76383,The Ten Commandments (1956 film),"Filmed on location in Egypt, Mount Sinai and t...",2
3,3,76383,76383,The Ten Commandments (1956 film),"In 1957, the film was nominated for seven Acad...",3
4,4,76383,76383,The Ten Commandments (1956 film),"In 1999, the film was selected for preservatio...",4
...,...,...,...,...,...,...
1166,1166,43376,43376,Boston Celtics,WFXT continued to carry the Fox network progra...,185
1167,1167,43376,43376,Boston Celtics,Section::::See also.\n,186
1168,1168,43376,43376,Boston Celtics,"BULLET::::- The Auerbach Center, official Celt...",187
1169,1169,43376,43376,Boston Celtics,BULLET::::- The Sports Museum (at TD Garden)\n,188


In [6]:
model = SentenceTransformer(MODEL, trust_remote_code=True)

`torch_dtype` is deprecated! Use `dtype` instead!
`torch_dtype` is deprecated! Use `dtype` instead!


In [7]:
index = faiss.read_index(INDEX)

In [ ]:
def retriev(item, index, k=5):

    task = "retrieval.query"
    query = model.encode(
        item['input'],
        task=task,
        prompt_name=task,
    )

    return index.search(np.array([query]), k)
    
retriev(df_q.iloc[1], index)



(array([[0.5026366, 0.9788302, 1.1069384, 1.1253469, 1.1774958]],
       dtype=float32),
 array([[947, 966, 948, 952, 980]]))

In [7]:
df_p.iloc[980].to_dict()

{'kilt_id': 1591817,
 'wikipedia_id': 1591817,
 'wikipedia_title': 'Adrienne Bailon',
 'paragraph': "Bailon got engaged to musician Israel Houghton on August 12, 2016 in Paris after six months of dating. They were married in Paris on November 11, 2016. Bailon became the step-mother to Israel's four children from his first marriage.\n",
 'paragraph_id': 33}